In [72]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

In [73]:
# Configurações do OpenAI
cliente = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
modelo = "gpt-4o-mini"

In [74]:
# Diretórios
BASE_DIR = os.getcwd()
DIR_PAI = os.path.dirname(BASE_DIR)
DIR_DATA = os.path.join(DIR_PAI, "data")
DIR_VISION = os.path.join(DIR_DATA, "outputs_vision")
DIR_DATA_PROCESSED_CLEAN =  os.path.join(DIR_DATA, "processed_clean")
DIR_DATA_REFINEMENT =  os.path.join(DIR_DATA, "outputs_vision_and_extractor")

In [75]:
# Função para carregar um JSON
def carregar_json(caminho_json):
    with open(caminho_json, "r", encoding="utf-8") as arquivo:
        return json.load(arquivo)

In [76]:
# Função para carregar um TXT
def carregar_txt(caminho_txt):
    with open(caminho_txt, "r", encoding="utf-8") as arquivo:
        return arquivo.read()

In [77]:
# Função para enviar o prompt para a OpenAI
def enviar_para_openai(contexto, descricao_txt):
    prompt = f"""
Você recebeu informações de dois formatos diferentes de arquivos sobre um mesmo conteúdo para analisar e unificar. 
Use os detalhes fornecidos no contexto (JSON) e na descrição do arquivo de visão (TXT). 
Combine as informações, elimine redundâncias, preencha lacunas e organize em formato claro e hierárquico. 
Preciso de um documento unificado com todas as informações técnicas disponíveis e suas unidades. 
**Descreva apenas com o conteúdo técnico necessário, seja objetivo e não faça nenhuma avaliação pessoal**

### Contexto JSON:
{contexto}

### Descrição TXT:
{descricao_txt}

Gere uma resposta detalhada e unificada das informações técnicas descritas nos documentos.
    """
    
    sistema = {
        "temperature": 0.3,
        "max_tokens": 1000,
        "top_p": 1
    }
    
    try:
        resposta = cliente.chat.completions.create(
            model=modelo,
            messages=[{"role": "user", "content": prompt}],
            **sistema,
        )
        return resposta.choices[0].message.content.strip()
    except Exception as e:
        print(f"Erro ao acessar a OpenAI: {e}")
        return None

In [78]:
# Processar os arquivos
def processar_arquivos():
    os.makedirs(DIR_DATA_REFINEMENT, exist_ok=True)
    arquivos_json = [f for f in os.listdir(DIR_DATA_PROCESSED_CLEAN) if f.endswith(".json")]

    for arquivo_json in arquivos_json:
        caminho_json = os.path.join(DIR_DATA_PROCESSED_CLEAN, arquivo_json)
        json_data = carregar_json(caminho_json)
        base_nome = os.path.splitext(arquivo_json)[0]
        subpasta_txt = os.path.join(DIR_VISION, base_nome)
        subpasta_saida = os.path.join(DIR_DATA_REFINEMENT, base_nome)

        # Criar a subpasta de saída correspondente
        os.makedirs(subpasta_saida, exist_ok=True)

        if not os.path.exists(subpasta_txt):
            print(f"Subpasta correspondente não encontrada: {subpasta_txt}")
            continue

        # Itera pelas páginas no JSON
        for pagina in json_data:
            numero_pagina = pagina["page"]
            nome_txt = f"{base_nome}_pag{numero_pagina}_description.txt"
            caminho_txt = os.path.join(subpasta_txt, nome_txt)

            if os.path.exists(caminho_txt):
                txt_data = carregar_txt(caminho_txt)
                resposta_openai = enviar_para_openai(pagina["text"], txt_data)

                # Salva a resposta na subpasta correspondente
                if resposta_openai:
                    saida_arquivo = os.path.join(subpasta_saida, f"{base_nome}_pag{numero_pagina}_resultado.json")
                    with open(saida_arquivo, "w", encoding="utf-8") as saida:
                        json.dump(
                            {"page": numero_pagina, "unified_analysis": resposta_openai},
                            saida,
                            ensure_ascii=False,
                            indent=4
                        )
                    print(f"Resultado salvo em: {saida_arquivo}")
            else:
                print(f"Arquivo TXT para página {numero_pagina} não encontrado: {nome_txt}")

In [79]:
# Executar o processamento
if __name__ == "__main__":
    processar_arquivos()

Resultado salvo em: c:\Users\axel.chepanski\doutor-ia\1 - extract-pdfs-transformer\data\outputs_transformers_vision_and_extractor\1 - Engine Control-Fuel System\1 - Engine Control-Fuel System_pag1_resultado.json
Resultado salvo em: c:\Users\axel.chepanski\doutor-ia\1 - extract-pdfs-transformer\data\outputs_transformers_vision_and_extractor\1 - Engine Control-Fuel System\1 - Engine Control-Fuel System_pag2_resultado.json
Resultado salvo em: c:\Users\axel.chepanski\doutor-ia\1 - extract-pdfs-transformer\data\outputs_transformers_vision_and_extractor\1 - Engine Control-Fuel System\1 - Engine Control-Fuel System_pag3_resultado.json
Resultado salvo em: c:\Users\axel.chepanski\doutor-ia\1 - extract-pdfs-transformer\data\outputs_transformers_vision_and_extractor\1 - Engine Control-Fuel System\1 - Engine Control-Fuel System_pag4_resultado.json
Resultado salvo em: c:\Users\axel.chepanski\doutor-ia\1 - extract-pdfs-transformer\data\outputs_transformers_vision_and_extractor\1 - Engine Control-Fue